In [78]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [79]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

In [80]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)


In [81]:
base['name'].value_counts()
base = base.drop('name', axis = 1)
base['seller'].value_counts()
base = base.drop('seller', axis = 1)
base['offerType'].value_counts()
base = base.drop('offerType', axis = 1)

In [82]:
i1 = base.loc[base.price <= 10]
base.price.mean()
base = base[base.price > 10]
i2 = base.loc[base.price > 350000]
base = base[base.price < 350000]

In [83]:
base.loc[pd.isnull(base['vehicleType'])]
base['vehicleType'].value_counts()
base.loc[pd.isnull(base['gearbox'])]
base['gearbox'].value_counts()
base.loc[pd.isnull(base['model'])]
base['model'].value_counts()
base.loc[pd.isnull(base['fuelType'])]
base['fuelType'].value_counts()
base.loc[pd.isnull(base['notRepairedDamage'])]
base['notRepairedDamage'].value_counts()

notRepairedDamage
nein    259301
ja       34004
Name: count, dtype: int64

In [84]:
valores = {'vehicleType': 'limousine',
           'gearbox': 'manuell',
           'model': 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)


In [85]:
previsores = base.iloc[:, 1:13].values
precoReal = base.iloc[:, 0].values


In [86]:
labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 10] = labelencoder_previsores.fit_transform(previsores[:, 10])

onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])],remainder='passthrough')

previsores = onehotencoder.fit_transform(previsores).toarray()

In [87]:
def criarRede(loss):
    regressor = Sequential()

    # Camadas da rede
    regressor.add(Dense(units=158, activation='relu', input_dim=316))
    regressor.add(Dense(units=158, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))

    # Compilação do modelo
    regressor.compile(optimizer='adam', loss=loss, metrics=['mean_absolute_error'])

    return regressor

In [88]:
regressor = KerasClassifier(
    model=criarRede, 
    epochs=100, 
    batch_size=300, 
    verbose=0  # Evitar muitos logs durante os testes
)

In [89]:
parametros = {'model__loss': ['mean_absolute_error',
                       'mean_squared_error',
                       'mean_absolute_percentage_error',
                       'mean_squared_logarithmic_error',
                       'squared_hinge']}

In [90]:
grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parametros,
                           scoring = 'neg_mean_absolute_error',
                           cv = 5)

In [91]:
grid_search = grid_search.fit(previsores, precoReal)

c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: User

In [92]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [94]:
previsoes = grid_search.best_estimator_.predict(previsores)

In [95]:
precoReal.mean()

5916.833945186492

In [96]:
previsoes.mean()

11.0